In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

In [2]:
df = pd.read_csv('./Human_Resources.csv')
df.head(5)

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [3]:
# dataframe size
df.shape

(1470, 35)

In [4]:
# column data types
df.dtypes

Age                          int64
Attrition                   object
BusinessTravel              object
DailyRate                    int64
Department                  object
DistanceFromHome             int64
Education                    int64
EducationField              object
EmployeeCount                int64
EmployeeNumber               int64
EnvironmentSatisfaction      int64
Gender                      object
HourlyRate                   int64
JobInvolvement               int64
JobLevel                     int64
JobRole                     object
JobSatisfaction              int64
MaritalStatus               object
MonthlyIncome                int64
MonthlyRate                  int64
NumCompaniesWorked           int64
Over18                      object
OverTime                    object
PercentSalaryHike            int64
PerformanceRating            int64
RelationshipSatisfaction     int64
StandardHours                int64
StockOptionLevel             int64
TotalWorkingYears   

In [5]:
# numerical and categorical columns
num_cols = df.columns[df.dtypes == 'int64']
cat_cols = df.columns[df.dtypes == 'object']

# checking if every column is included 
print(len(num_cols) + len(cat_cols) == len(df.columns))

True


Encoding:
- One Hot: Department, MaritalStatus, Job Role,Gender, OverTime, EducationField
- Ordinal: BusinessTravel

So we will do data transformations two ways when it comes to encoding: 
1. For general classification Models
2. For tree based models

In [6]:
# 1. Check columns that can be dropped
cols_drop = []
for col in cat_cols:
    if len(df[col].unique()) == 1:
        cols_drop.append(col)

cols_drop

['Over18']

In [7]:
df['StandardHours'].unique()

array([80], dtype=int64)

Columns to drop: Over18, StandardHours and EmployeeNumber

In [8]:
df.drop(['Over18','StandardHours','EmployeeNumber'],axis = 1,inplace= True)
df.shape

(1470, 32)

In [9]:
# 2. Checking Missing Values
df.isnull().sum().sum()

0

So there are no missing values. <br>
Now we will be using non-tree based models like Logistic Regression and SVC, then tree based models like Random forest, XGBoost and Catboost. Hence the encoding and scaling will be different. <br>
Now we will start by splittng the data into train and test sets, with a 80:20 ratio. The train set will also be used for doing cross valaidation to tune hyperparameters and get the best model.
And then a final result on the test set.

In [10]:
# Making the input data matrix and target vector
X = df.drop('Attrition',axis=1)
y = df['Attrition'].map({
    'Yes': 1,
    'No': 0
})
X.shape, y.shape

((1470, 31), (1470,))

In [11]:
# Creating the train and test dataset
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test = train_test_split(
    X,y,test_size=0.2,stratify=y,random_state=42
)

In [12]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((1176, 31), (294, 31), (1176,), (294,))

In [13]:
X_train

,Age,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EnvironmentSatisfaction,Gender,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
1194,47,Travel_Rarely,1225,Sales,2,4,Life Sciences,1,2,Female,...,3,3,3,29,2,3,3,2,1,2
128,22,Travel_Rarely,594,Research & Development,2,1,Technical Degree,1,3,Male,...,3,3,1,3,2,3,2,1,2,1
810,46,Travel_Rarely,406,Sales,3,1,Marketing,1,1,Male,...,3,4,1,23,3,3,12,9,4,9
478,25,Travel_Rarely,622,Sales,13,1,Medical,1,2,Male,...,3,3,0,7,1,3,7,4,0,6
491,43,Travel_Frequently,1001,Research & Development,9,5,Medical,1,4,Male,...,3,2,1,10,3,3,8,7,4,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1213,23,Travel_Rarely,427,Sales,7,3,Life Sciences,1,3,Male,...,4,2,1,3,2,3,3,2,0,2
963,38,Travel_Rarely,1009,Sales,2,2,Life Sciences,1,2,Female,...,3,4,1,11,3,3,7,7,1,7
734,22,Travel_Rarely,217,Research & Development,8,1,Life Sciences,1,2,Male,...,3,1,1,4,3,2,4,3,1,1
1315,36,Travel_Rarely,430,Research & Development,2,4,Other,1,4,Female,...,4,4,1,15,2,3,1,0,0,0


In [66]:
sum = 0
for y in y_train:
    sum += y

print(sum)


190


In [67]:
y_train

1194    0
128     0
810     0
478     0
491     0
       ..
1213    1
963     0
734     0
1315    0
1292    0
Name: Attrition, Length: 1176, dtype: int64

In [14]:
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,StandardScaler

In [ ]:
ohe_cols = ['Department','EducationField','JobRole','MaritalStatus','Gender','OverTime']
ord_cols = ['BusinessTravel']
num_cols = num_cols.difference(['EmployeeCount', 'StandardHours'])

In [16]:
ordinal_encoder = OrdinalEncoder(categories=[['Non-Travel','Travel_Rarely', 'Travel_Frequently']])
X_train[ord_cols] = ordinal_encoder.fit_transform(X_train[ord_cols])
X_test[ord_cols] = ordinal_encoder.transform(X_test[ord_cols])
X_train[ord_cols], X_test[ord_cols]

(      BusinessTravel
 1194             1.0
 128              1.0
 810              1.0
 478              1.0
 491              2.0
 ...              ...
 1213             1.0
 963              1.0
 734              1.0
 1315             1.0
 1292             2.0
 
 [1176 rows x 1 columns],
       BusinessTravel
 1061             0.0
 891              1.0
 456              1.0
 922              1.0
 69               1.0
 ...              ...
 1269             1.0
 1352             1.0
 1236             1.0
 1023             1.0
 285              1.0
 
 [294 rows x 1 columns])

In [17]:
ohe = OneHotEncoder(sparse_output=False,handle_unknown='ignore')
ohe_train_array = ohe.fit_transform(X_train[ohe_cols])
# Transform X_test using the already-fitted encoder
ohe_test_array = ohe.transform(X_test[ohe_cols])

In [19]:
feature_names = ohe.get_feature_names_out(ohe_cols)
print("Feature names:", feature_names)

Feature names: ['Department_Human Resources' 'Department_Research & Development'
 'Department_Sales' 'EducationField_Human Resources'
 'EducationField_Life Sciences' 'EducationField_Marketing'
 'EducationField_Medical' 'EducationField_Other'
 'EducationField_Technical Degree' 'JobRole_Healthcare Representative'
 'JobRole_Human Resources' 'JobRole_Laboratory Technician'
 'JobRole_Manager' 'JobRole_Manufacturing Director'
 'JobRole_Research Director' 'JobRole_Research Scientist'
 'JobRole_Sales Executive' 'JobRole_Sales Representative'
 'MaritalStatus_Divorced' 'MaritalStatus_Married' 'MaritalStatus_Single'
 'Gender_Female' 'Gender_Male' 'OverTime_No' 'OverTime_Yes']


In [21]:
ohe_train_df = pd.DataFrame(ohe_train_array,columns=feature_names,index=X_train.index)
ohe_test_df = pd.DataFrame(ohe_test_array,columns=feature_names,index=X_test.index)


In [22]:
ohe_train_df.head(5)

,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,JobRole_Healthcare Representative,...,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Gender_Female,Gender_Male,OverTime_No,OverTime_Yes
1194,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
128,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
810,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
478,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
491,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0


In [23]:
ohe_test_df.head(5)

,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,EducationField_Marketing,EducationField_Medical,EducationField_Other,EducationField_Technical Degree,JobRole_Healthcare Representative,...,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Gender_Female,Gender_Male,OverTime_No,OverTime_Yes
1061,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
891,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
456,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
922,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0
69,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0


In [24]:
X_train = pd.concat([X_train.drop(columns=ohe_cols),ohe_train_df],axis = 1)
X_test = pd.concat([X_test.drop(columns=ohe_cols),ohe_test_df],axis = 1)
X_train.columns, X_test.columns

(Index(['Age', 'BusinessTravel', 'DailyRate', 'DistanceFromHome', 'Education',
        'EmployeeCount', 'EnvironmentSatisfaction', 'HourlyRate',
        'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MonthlyIncome',
        'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike',
        'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel',
        'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
        'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
        'YearsWithCurrManager', 'Department_Human Resources',
        'Department_Research & Development', 'Department_Sales',
        'EducationField_Human Resources', 'EducationField_Life Sciences',
        'EducationField_Marketing', 'EducationField_Medical',
        'EducationField_Other', 'EducationField_Technical Degree',
        'JobRole_Healthcare Representative', 'JobRole_Human Resources',
        'JobRole_Laboratory Technician', 'JobRole_Manager',
        'JobRole_Manufacturi

In [37]:
# Scaling the numerical columns/features
scaler = StandardScaler()
X_train[num_cols] = scaler.fit_transform(X_train[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])

In [38]:
X_train[num_cols]

,Age,DailyRate,DistanceFromHome,Education,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,JobSatisfaction,MonthlyIncome,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
1194,1.090194,1.049455,-0.899915,1.064209,-0.658710,-0.908436,1.795282,1.762189,-0.647997,2.026752,...,-0.432065,0.240218,2.613100,2.261482,-0.605389,0.337621,-0.665706,-0.625365,-0.368024,-0.616406
128,-1.634828,-0.523449,-0.899915,-1.855332,0.260202,1.694111,0.373564,-0.986265,1.153526,-0.864408,...,-0.432065,0.240218,0.247430,-1.072675,-0.605389,0.337621,-0.830071,-0.905635,-0.056884,-0.897047
810,0.981193,-0.992080,-0.777610,-1.855332,-1.577622,-0.662913,0.373564,1.762189,0.252765,2.347706,...,-0.432065,1.160403,0.247430,1.492061,0.190962,0.337621,0.813578,1.336527,0.565398,1.348076
478,-1.307825,-0.453653,0.445433,-1.855332,-0.658710,-1.252169,0.373564,-0.986265,0.252765,-0.956202,...,-0.432065,0.240218,-0.935405,-0.559727,-1.401740,0.337621,-0.008246,-0.064824,-0.679165,0.506155
491,0.654191,0.491086,-0.043784,2.037390,1.179114,0.319180,0.373564,-0.070114,0.252765,-0.185956,...,-0.432065,-0.679966,0.247430,-0.175017,0.190962,0.337621,0.156119,0.775986,0.565398,0.786795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1213,-1.525827,-0.939733,-0.288393,0.091029,0.260202,1.645007,0.373564,-0.986265,1.153526,-0.917721,...,2.314467,-0.679966,0.247430,-1.072675,-0.605389,0.337621,-0.665706,-0.625365,-0.679165,-0.616406
963,0.109186,0.511028,-0.899915,-0.882152,-0.658710,-1.694111,0.373564,-0.070114,-1.548758,0.075020,...,-0.432065,1.160403,0.247430,-0.046780,0.190962,0.337621,-0.008246,0.775986,-0.368024,0.786795
734,-1.634828,-1.463203,-0.166088,-1.855332,-0.658710,1.399483,-2.469873,-0.986265,-1.548758,-0.879886,...,-0.432065,-1.600151,0.247430,-0.944438,0.190962,-1.055510,-0.501341,-0.345095,-0.368024,-0.897047
1315,-0.108815,-0.932255,-0.899915,1.064209,1.179114,0.368285,0.373564,-0.070114,-0.647997,0.089853,...,2.314467,1.160403,0.247430,0.466167,-0.605389,0.337621,-0.994436,-1.185905,-0.679165,-1.177687


In [39]:
X_train.shape,y_train.shape

((1176, 50), (1176,))

In [ ]:
# Now we will 

# Model Training

In [82]:
from sklearn.model_selection import GridSearchCV, cross_val_predict, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import make_scorer, recall_score, classification_report

In [80]:
# For attrition detection, recall for class 1 (quitters) is most important
# This custom scorer focuses on recall for the positive class (employees who quit)
scoring = make_scorer(recall_score, pos_label=1)

In [81]:
models_params = {
    'Logistic Regression': (LogisticRegression(max_iter=1000,random_state=42), {
        'penalty': ['l1', 'l2'],
        'C': [0.01, 0.1, 1, 10],
        'solver': ['liblinear']
    }),
    'KNN': (KNeighborsClassifier(), {
        'n_neighbors': [3, 5, 7],
        'weights': ['uniform', 'distance'],
        'metric': ['euclidean', 'manhattan']
    }),
    'SVC': (SVC(), {
        'C': [0.1, 1, 10],
        'kernel': ['linear', 'rbf'],
        'gamma': ['scale', 'auto']
    }),
    'LDA': (
    LinearDiscriminantAnalysis(),
    [
        {'solver': ['svd']},  # no shrinkage
        {'solver': ['lsqr'], 'shrinkage': [None, 'auto']}  # shrinkage valid here
    ]
)
}

In [83]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [84]:
best_models = {}
results_summary = []

In [ ]:
# for name, (model, params) in models_params.items():
#     print(f"Training {name}...")
    
#     # Grid search with recall scoring (for detecting quitters)
#     grid = GridSearchCV(model, params, cv=cv, scoring=scoring, n_jobs=-1)
#     grid.fit(X_train, y_train)
#     best_models[name] = grid
    
#     # Get predictions on test set
#     y_pred = grid.predict(X_test)
    
#     # Generate classification report
#     report = classification_report(y_test, y_pred, output_dict=True)
    
#     print(f"\n{name} Results:")
#     print(f"Best CV Recall Score (Class 1): {grid.best_score_:.4f}")
#     print(f"Best Params: {grid.best_params_}")
#     print("\nClassification Report:")
#     print(classification_report(y_test, y_pred))
    
#     # Extract key metrics for comparison
#     recall_class_1 = report['1']['recall']  # This is what you want to maximize
#     precision_class_1 = report['1']['precision']
#     f1_class_1 = report['1']['f1-score']
    
#     results_summary.append({
#         'Model': name,
#         'Recall_Class_1_Quitters': recall_class_1,
#         'Precision_Class_1_Quitters': precision_class_1,
#         'F1_Class_1_Quitters': f1_class_1,
#         'Overall_Accuracy': report['accuracy']
#     })
    
#     print(f"Key Metrics for Attrition Detection:")
#     print(f"- Recall for Class 1 (Quitters): {recall_class_1:.3f}")
#     print(f"- Precision for Class 1 (Quitters): {precision_class_1:.3f}")
#     print(f"- F1 for Class 1 (Quitters): {f1_class_1:.3f}")
#     print("-" * 60)

# # Summary comparison
# print("\nSUMMARY - Model Comparison for Employee Attrition:")
# df_results = pd.DataFrame(results_summary)
# df_results = df_results.sort_values('Recall_Class_1_Quitters', ascending=False)
# print(df_results.round(3))

# print(f"\nBest Model for Attrition Detection: {df_results.iloc[0]['Model']}")
# print(f"Best Recall for detecting quitters: {df_results.iloc[0]['Recall_Class_1_Quitters']:.3f}")

In [ ]:
# for name, (model, params) in models_params.items():
#     print(f"\nTraining {name}...")
    
#     # Grid search with 5-fold CV on TRAINING data only
#     grid = GridSearchCV(model, params, cv=cv, scoring=scoring, n_jobs=-1)
#     grid.fit(X_train, y_train)
#     best_models[name] = grid
    
#     # Get cross-validation predictions on TRAINING data
#     # This gives us unbiased estimates without touching test data
#     cv_pred = cross_val_predict(grid.best_estimator_, X_train, y_train, cv=5)
    
#     # Classification report based on CV predictions (TRAINING DATA)
#     cv_report = classification_report(y_train, cv_pred, output_dict=True)
    
#     print(f"Best CV Recall Score (Class 1): {grid.best_score_:.4f}")
#     print(f"Best Params: {grid.best_params_}")
#     print("\nCross-Validation Classification Report (Training Data):")
#     print(classification_report(y_train, cv_pred))
    
#     # Extract key metrics for model selection
#     recall_class_1 = cv_report['1']['recall']
#     precision_class_1 = cv_report['1']['precision'] 
#     f1_class_1 = cv_report['1']['f1-score']
    
#     cv_results_summary.append({
#         'Model': name,
#         'CV_Recall_Class_1': recall_class_1,
#         'CV_Precision_Class_1': precision_class_1,
#         'CV_F1_Class_1': f1_class_1,
#         'CV_Overall_Accuracy': cv_report['accuracy'],
#         'GridSearch_Best_Score': grid.best_score_
#     })
    
#     print(f"Cross-Validation Metrics for Attrition Detection:")
#     print(f"- CV Recall for Class 1 (Quitters): {recall_class_1:.3f}")
#     print(f"- CV Precision for Class 1 (Quitters): {precision_class_1:.3f}")
#     print(f"- CV F1 for Class 1 (Quitters): {f1_class_1:.3f}")
#     print("-" * 50)

# # Model comparison and selection based on TRAINING data performance
# print("\n" + "="*60)
# print("MODEL SELECTION SUMMARY (Based on Training Data CV)")
# print("="*60)
# df_cv_results = pd.DataFrame(cv_results_summary)
# df_cv_results = df_cv_results.sort_values('CV_Recall_Class_1', ascending=False)
# print(df_cv_results.round(3))

# # Select best model
# best_model_name = df_cv_results.iloc[0]['Model']
# best_model = best_models[best_model_name]

# print(f"\nSELECTED BEST MODEL: {best_model_name}")
# print(f"Expected Recall for detecting quitters: {df_cv_results.iloc[0]['CV_Recall_Class_1']:.3f}")

# print("\n" + "="*60)
# print("STEP 2: FINAL EVALUATION ON TEST DATA (ONCE ONLY)")
# print("="*60)

# # STEP 2: FINAL EVALUATION ON TEST DATA (ONLY ONCE!)
# print(f"\nEvaluating selected model ({best_model_name}) on unseen test data...")

# # Final predictions on test data
# final_predictions = best_model.predict(X_test)

# # Final classification report
# print(f"\nFINAL TEST RESULTS for {best_model_name}:")
# print("="*40)
# print(classification_report(y_test, final_predictions))

# # Extract final test metrics
# final_report = classification_report(y_test, final_predictions, output_dict=True)
# final_recall_class_1 = final_report['1']['recall']
# final_precision_class_1 = final_report['1']['precision']
# final_f1_class_1 = final_report['1']['f1-score']

# print(f"\nFINAL PERFORMANCE SUMMARY:")
# print(f"- Test Recall for Class 1 (Quitters): {final_recall_class_1:.3f}")
# print(f"- Test Precision for Class 1 (Quitters): {final_precision_class_1:.3f}")
# print(f"- Test F1 for Class 1 (Quitters): {final_f1_class_1:.3f}")

# print(f"\nModel Generalization Check:")
# expected_recall = df_cv_results.iloc[0]['CV_Recall_Class_1']
# print(f"- Expected Recall (CV): {expected_recall:.3f}")
# print(f"- Actual Test Recall: {final_recall_class_1:.3f}")
# print(f"- Difference: {abs(expected_recall - final_recall_class_1):.3f}")

# if abs(expected_recall - final_recall_class_1) < 0.05:
#     print("✓ Model generalizes well!")
# else:
#     print("⚠ Model may be overfitting - consider more regularization")

# print("\n" + "="*60)
# print("DEPLOYMENT READY MODEL")
# print("="*60)
# print(f"Selected Model: {best_model_name}")
# print(f"Model Object: best_models['{best_model_name}']")
# print("This model is now ready for production deployment.")

In [85]:

for name, (model, params) in models_params.items():
    print(f"\n{name}:")
    print("-" * 30)
    
    # GridSearchCV with 5-fold CV, optimizing for recall of class 1
    grid = GridSearchCV(model, params, cv=cv, scoring=scoring, n_jobs=-1)
    grid.fit(X_train, y_train)
    best_models[name] = grid
    
    # Get predictions on training data using the best model from grid search
    best_model = grid.best_estimator_
    y_train_pred = best_model.predict(X_train)
    
    # Classification report for this model (training data)
    print(f"Best CV Recall Score for Class 1 (Quitters): {grid.best_score_:.4f}")
    print(f"Best Parameters: {grid.best_params_}")
    print("\nClassification Report (Training Data):")
    print(classification_report(y_train, y_train_pred))
    
    # Extract key metrics for comparison
    train_report = classification_report(y_train, y_train_pred, output_dict=True)
    recall_class_1 = train_report['1']['recall']
    precision_class_1 = train_report['1']['precision']
    f1_class_1 = train_report['1']['f1-score']
    
    results_summary.append({
        'Model': name,
        'CV_Recall_Score': grid.best_score_,
        'Train_Recall_Class_1': recall_class_1,
        'Train_Precision_Class_1': precision_class_1,
        'Train_F1_Class_1': f1_class_1
    })

# Compare all models and select the best one
print("\n" + "="*70)
print("MODEL COMPARISON SUMMARY")
print("="*70)
df_results = pd.DataFrame(results_summary)
df_results = df_results.sort_values('CV_Recall_Score', ascending=False)
print(df_results.round(4))

# Select best model based on CV recall score
best_model_name = df_results.iloc[0]['Model']
best_model = best_models[best_model_name]

print(f"\nBEST MODEL SELECTED: {best_model_name}")
print(f"CV Recall for Class 1 (Quitters): {df_results.iloc[0]['CV_Recall_Score']:.4f}")

# STEP 2: Final evaluation on test data (ONCE ONLY)
print("\n" + "="*70)
print("STEP 2: Final Test Evaluation (ONCE ONLY - NO FURTHER CHANGES ALLOWED)")
print("="*70)

# Final predictions on test data
y_test_pred = best_model.predict(X_test)

print(f"\nFinal Test Results for {best_model_name}:")
print(classification_report(y_test, y_test_pred))

# Extract the key metric you care about
test_report = classification_report(y_test, y_test_pred, output_dict=True)
test_recall_class_1 = test_report['1']['recall']

print(f"\nKEY RESULT:")
print(f"Test Recall for Class 1 (Employees who quit): {test_recall_class_1:.4f}")
print(f"This means your model catches {test_recall_class_1*100:.1f}% of employees who actually quit.")

print(f"\nModel is ready for deployment: best_models['{best_model_name}']")


Logistic Regression:
------------------------------
Best CV Recall Score for Class 1 (Quitters): 0.4526
Best Parameters: {'C': 1, 'penalty': 'l2', 'solver': 'liblinear'}

Classification Report (Training Data):
              precision    recall  f1-score   support

           0       0.91      0.98      0.94       986
           1       0.84      0.50      0.63       190

    accuracy                           0.90      1176
   macro avg       0.88      0.74      0.79      1176
weighted avg       0.90      0.90      0.89      1176


KNN:
------------------------------
Best CV Recall Score for Class 1 (Quitters): 0.2000
Best Parameters: {'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'uniform'}

Classification Report (Training Data):
              precision    recall  f1-score   support

           0       0.90      0.99      0.94       986
           1       0.87      0.44      0.58       190

    accuracy                           0.90      1176
   macro avg       0.89      0.71 

In [ ]:
# # STEP 1: MODEL SELECTION AND EVALUATION (TRAINING DATA ONLY)
# # For attrition detection, recall for class 1 (quitters) is most important
# scoring = make_scorer(recall_score, pos_label=1)

# models_params = {
#     'Logistic Regression': (
#         LogisticRegression(max_iter=1000, random_state=42), {
#             'penalty': ['l1', 'l2'],
#             'C': [0.01, 0.1, 1, 10],
#             'solver': ['liblinear']
#         }
#     ),
#     'KNN': (
#         KNeighborsClassifier(), {
#             'n_neighbors': [3, 5, 7],
#             'weights': ['uniform', 'distance'],
#             'metric': ['euclidean', 'manhattan']
#         }
#     ),
#     'SVC': (
#         SVC(), {
#             'C': [0.1, 1, 10],
#             'kernel': ['linear', 'rbf'],
#             'gamma': ['scale', 'auto']
#         }
#     ),
#     'LDA': (
#         LinearDiscriminantAnalysis(), {
#             'solver': ['svd', 'lsqr'],
#             'shrinkage': [None, 'auto']
#         }
#     ),
#     'Naive Bayes': (
#         GaussianNB(), {
#             'var_smoothing': [1e-9, 1e-8, 1e-7]
#         }
#     )
# }

# print("="*60)
# print("STEP 1: MODEL SELECTION (USING ONLY TRAINING DATA)")
# print("="*60)

# best_models = {}
# cv_results_summary = []

# for name, (model, params) in models_params.items():
#     print(f"\nTraining {name}...")
    
#     # Grid search with 5-fold CV on TRAINING data only
#     grid = GridSearchCV(model, params, cv=5, scoring=scoring, n_jobs=-1)
#     grid.fit(X_train, y_train)
#     best_models[name] = grid
    
#     # Get cross-validation predictions on TRAINING data
#     # This gives us unbiased estimates without touching test data
#     cv_pred = cross_val_predict(grid.best_estimator_, X_train, y_train, cv=5)
    
#     # Classification report based on CV predictions (TRAINING DATA)
#     cv_report = classification_report(y_train, cv_pred, output_dict=True)
    
#     print(f"Best CV Recall Score (Class 1): {grid.best_score_:.4f}")
#     print(f"Best Params: {grid.best_params_}")
#     print("\nCross-Validation Classification Report (Training Data):")
#     print(classification_report(y_train, cv_pred))
    
#     # Extract key metrics for model selection
#     recall_class_1 = cv_report['1']['recall']
#     precision_class_1 = cv_report['1']['precision'] 
#     f1_class_1 = cv_report['1']['f1-score']
    
#     cv_results_summary.append({
#         'Model': name,
#         'CV_Recall_Class_1': recall_class_1,
#         'CV_Precision_Class_1': precision_class_1,
#         'CV_F1_Class_1': f1_class_1,
#         'CV_Overall_Accuracy': cv_report['accuracy'],
#         'GridSearch_Best_Score': grid.best_score_
#     })
    
#     print(f"Cross-Validation Metrics for Attrition Detection:")
#     print(f"- CV Recall for Class 1 (Quitters): {recall_class_1:.3f}")
#     print(f"- CV Precision for Class 1 (Quitters): {precision_class_1:.3f}")
#     print(f"- CV F1 for Class 1 (Quitters): {f1_class_1:.3f}")
#     print("-" * 50)

# # Model comparison and selection based on TRAINING data performance
# print("\n" + "="*60)
# print("MODEL SELECTION SUMMARY (Based on Training Data CV)")
# print("="*60)
# df_cv_results = pd.DataFrame(cv_results_summary)
# df_cv_results = df_cv_results.sort_values('CV_Recall_Class_1', ascending=False)
# print(df_cv_results.round(3))

# # Select best model
# best_model_name = df_cv_results.iloc[0]['Model']
# best_model = best_models[best_model_name]

# print(f"\nSELECTED BEST MODEL: {best_model_name}")
# print(f"Expected Recall for detecting quitters: {df_cv_results.iloc[0]['CV_Recall_Class_1']:.3f}")

# print("\n" + "="*60)
# print("STEP 2: FINAL EVALUATION ON TEST DATA (ONCE ONLY)")
# print("="*60)

# # STEP 2: FINAL EVALUATION ON TEST DATA (ONLY ONCE!)
# print(f"\nEvaluating selected model ({best_model_name}) on unseen test data...")

# # Final predictions on test data
# final_predictions = best_model.predict(X_test)

# # Final classification report
# print(f"\nFINAL TEST RESULTS for {best_model_name}:")
# print("="*40)
# print(classification_report(y_test, final_predictions))

# # Extract final test metrics
# final_report = classification_report(y_test, final_predictions, output_dict=True)
# final_recall_class_1 = final_report['1']['recall']
# final_precision_class_1 = final_report['1']['precision']
# final_f1_class_1 = final_report['1']['f1-score']

# print(f"\nFINAL PERFORMANCE SUMMARY:")
# print(f"- Test Recall for Class 1 (Quitters): {final_recall_class_1:.3f}")
# print(f"- Test Precision for Class 1 (Quitters): {final_precision_class_1:.3f}")
# print(f"- Test F1 for Class 1 (Quitters): {final_f1_class_1:.3f}")

# print(f"\nModel Generalization Check:")
# expected_recall = df_cv_results.iloc[0]['CV_Recall_Class_1']
# print(f"- Expected Recall (CV): {expected_recall:.3f}")
# print(f"- Actual Test Recall: {final_recall_class_1:.3f}")
# print(f"- Difference: {abs(expected_recall - final_recall_class_1):.3f}")

# if abs(expected_recall - final_recall_class_1) < 0.05:
#     print("✓ Model generalizes well!")
# else:
#     print("⚠ Model may be overfitting - consider more regularization")

# print("\n" + "="*60)
# print("DEPLOYMENT READY MODEL")
# print("="*60)
# print(f"Selected Model: {best_model_name}")
# print(f"Model Object: best_models['{best_model_name}']")
# print("This model is now ready for production deployment.")

In [73]:
for name, (model, params) in models_params.items():
    grid = GridSearchCV(model, params, cv=5, scoring=scoring, n_jobs=-1)
    grid.fit(X_train, y_train)
    best_models[name] = grid
    print(f"{name}: Best recall Score = {grid.best_score_:.4f}")
    print(f"Best Params: {grid.best_params_}\n")


Logistic Regression: Best recall Score = 0.4421
Best Params: {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}

KNN: Best recall Score = 0.2158
Best Params: {'metric': 'euclidean', 'n_neighbors': 3, 'weights': 'uniform'}

SVC: Best recall Score = 0.4263
Best Params: {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}

LDA: Best recall Score = 0.4211
Best Params: {'solver': 'svd'}



In [ ]:
X_train['BusinessTravel'].unique()

In [ ]:
X_train['Department'].unique()

In [ ]:
X_train['EducationField'].unique()

In [ ]:
X_train['JobRole'].unique()

In [ ]:
X_train['Gender'].unique()

In [ ]:
X_train['MaritalStatus'].unique()

In [ ]:
X_train['OverTime'].unique()

In [ ]:
df['Employee']

0       1
1       1
2       1
3       1
4       1
       ..
1465    1
1466    1
1467    1
1468    1
1469    1
Name: EmployeeCount, Length: 1470, dtype: int64